# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dropout, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11130985716984216142
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 13630989628875700755
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# load English data
english_sentences = helper.load_data('data/small_vocab_en')

# load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset loaded.')

Dataset loaded.


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
# test if the 2 datasets have the same length
len(english_sentences) == len(french_sentences)

True

In [6]:
for i in range(2):
    print(f'small_vocab_en line {i+1}: {english_sentences[i]}')
    print(f'small_vocab_fr line {i+1}: {french_sentences[i]}')

small_vocab_en line 1: new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr line 1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en line 2: the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr line 2: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print(f'{len([word for sentence in english_sentences for word in sentence.split()])} English words.')
print(f'{len(english_words_counter)} unique English words.')
print(f'10 most commmon words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print('\n')
print(f'{len([word for sentence in french_sentences for word in sentence.split()])} French words.')
print(f'{len(french_words_counter)} unique French words.')
print(f'10 most commmon words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 most commmon words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"


1961295 French words.
355 unique French words.
10 most commmon words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    token = Tokenizer()
    token.fit_on_texts(x)
    
    return token.texts_to_sequences(x), token

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)

print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(seq) for seq in x])
    
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """Preprocess x and y.
    :param x: feature list of sentences.
    :param y: label list of sentences.
    :return: tuple of (preprocessed x, preprocessed y, x tokenizer, y tokenizer)
    """
    
    # tokenize
    preproc_x, x_tk = tokenize(x)
    preproc_y, y_tk = tokenize(y)
    
    # padding
    preproc_x = pad(preproc_x)
    preproc_y = pad(preproc_y)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preproc_y = preproc_y.reshape(*preproc_y.shape, 1)
    
    return preproc_x, preproc_y, x_tk, y_tk

In [11]:
preproc_en_sentences, preproc_fr_sentences, en_token, fr_token = preprocess(english_sentences, french_sentences)

max_en_seq_len = preproc_en_sentences.shape[1]
max_fr_seq_len = preproc_fr_sentences.shape[1]
en_vocab_size = len(en_token.word_index)
fr_vocab_size = len(fr_token.word_index)

print('Data preprocessed.')
print(f'Max English sentence length: {max_en_seq_len}')
print(f'Max French sentence length: {max_fr_seq_len}')
print(f'English vocabulary size: {en_vocab_size}')
print(f'French vocabulary size: {fr_vocab_size}')

Data preprocessed.
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [12]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    
    index_to_words = {id: word for (word, id) in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, axis=1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [20]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    # model architecture
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # compile model
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

In [21]:
# reshaping the input to work with a basic RNN
tmp_x = pad(preproc_en_sentences, max_fr_seq_len)
tmp_x = tmp_x.reshape((-1, preproc_fr_sentences.shape[-2], 1))

# train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_fr_seq_len,
    en_vocab_size,
    fr_vocab_size
)

# model summary
print(simple_rnn_model.summary())

simple_rnn_model.fit(
    tmp_x, 
    preproc_fr_sentences, 
    batch_size=1024, 
    epochs=10, 
    validation_split=0.2
)

# print prediction
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], fr_token))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 21, 256)           264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 344)           88408     
Total params: 352,600
Trainable params: 352,600
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 119us/step - loss: 2.7093 - acc: 0.4588 - val_loss: nan - val_acc: 0.5371
Epoch 2/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.8049 - acc: 0.5527 - val_loss: nan - val_acc: 0.5983
Epoch 3/10
110288/110288 [==============================] - 13s 114u

In [48]:
print('Original English sentence: ')
print(english_sentences[:1][0])
print('\n')
print('Model translation: ')
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], fr_token))
print('\n')
print('Correct French translation: ')
print(french_sentences[:1][0])

Original English sentence: 
new jersey is sometimes quiet during autumn , and it is snowy in april .


Model translation: 
new jersey est parfois calme en l' et il est il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Correct French translation: 
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [36]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # model architecture        
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size, 
                        output_dim=256, 
                        input_length=input_shape[1], 
                        input_shape=input_shape[1:]))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # compile model
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)

In [16]:
# reshaping the input
tmp_x = pad(preproc_en_sentences, max_fr_seq_len)
tmp_x = tmp_x.reshape((-1, preproc_fr_sentences.shape[-2]))

In [38]:
# train the neural network
embed_model = embed_model(
    tmp_x.shape,
    max_fr_seq_len,
    en_vocab_size,
    fr_vocab_size
)

embed_model.fit(
    tmp_x, 
    preproc_fr_sentences, 
    batch_size=1024, 
    epochs=10, 
    validation_split=0.2
)

# print prediction
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], fr_token))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 17s 156us/step - loss: 3.0836 - acc: 0.4450 - val_loss: nan - val_acc: 0.4965
Epoch 2/10
110288/110288 [==============================] - 16s 148us/step - loss: 1.9249 - acc: 0.5506 - val_loss: nan - val_acc: 0.6345
Epoch 3/10
110288/110288 [==============================] - 16s 149us/step - loss: 1.3334 - acc: 0.6601 - val_loss: nan - val_acc: 0.7337
Epoch 4/10
110288/110288 [==============================] - 16s 149us/step - loss: 0.9457 - acc: 0.7505 - val_loss: nan - val_acc: 0.8077
Epoch 5/10
110288/110288 [==============================] - 16s 148us/step - loss: 0.7090 - acc: 0.8013 - val_loss: nan - val_acc: 0.8367
Epoch 6/10
110288/110288 [==============================] - 16s 148us/step - loss: 0.5886 - acc: 0.8282 - val_loss: nan - val_acc: 0.8541
Epoch 7/10
110288/110288 [==============================] - 16s 149us/step - loss: 0.5141 - acc: 0.8476 - val_loss: nan -

In [41]:
embed_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 21, 256)           50944     
_________________________________________________________________
lstm_11 (LSTM)               (None, 21, 256)           525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 21, 344)           88408     
Total params: 664,664
Trainable params: 664,664
Non-trainable params: 0
_________________________________________________________________


In [44]:
print('Original English sentence: ')
print(english_sentences[:1][0])
print('\n')
print('Model translation: ')
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], fr_token))
print('\n')
print('Correct French translation: ')
print(french_sentences[:1][0])

Original English sentence: 
new jersey is sometimes quiet during autumn , and it is snowy in april .


Model translation: 
new jersey est parfois calme en l' et il est neigeux neigeux en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Correct French translation: 
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [45]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # model architecture
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True), 
                            input_shape=input_shape[1:]))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # compile model
    learning_rate = 1e-3    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

In [42]:
# reshaping the input
tmp_x = pad(preproc_en_sentences, max_fr_seq_len)
tmp_x = tmp_x.reshape((-1, preproc_fr_sentences.shape[-2], 1))

In [46]:
# TODO: Train and Print prediction(s)
# train the neural network
bd_model = bd_model(
    tmp_x.shape,
    max_fr_seq_len,
    en_vocab_size,
    fr_vocab_size
)

bd_model.fit(
    tmp_x, 
    preproc_fr_sentences, 
    batch_size=1024, 
    epochs=10, 
    validation_split=0.2
)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 25s 222us/step - loss: 2.2741 - acc: 0.5160 - val_loss: nan - val_acc: 0.6007
Epoch 2/10
110288/110288 [==============================] - 21s 194us/step - loss: 1.4759 - acc: 0.6017 - val_loss: nan - val_acc: 0.6349
Epoch 3/10
110288/110288 [==============================] - 21s 195us/step - loss: 1.3210 - acc: 0.6269 - val_loss: nan - val_acc: 0.6481
Epoch 4/10
110288/110288 [==============================] - 21s 195us/step - loss: 1.2477 - acc: 0.6382 - val_loss: nan - val_acc: 0.6625
Epoch 5/10
110288/110288 [==============================] - 22s 195us/step - loss: 1.1980 - acc: 0.6478 - val_loss: nan - val_acc: 0.6667
Epoch 6/10
110288/110288 [==============================] - 21s 195us/step - loss: 1.1585 - acc: 0.6557 - val_loss: nan - val_acc: 0.6807
Epoch 7/10
110288/110288 [==============================] - 21s 195us/step - loss: 1.1234 - acc: 0.6629 - val_loss: nan -

In [47]:
bd_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_13 (Bidirectio (None, 21, 512)           528384    
_________________________________________________________________
dropout_4 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           176472    
Total params: 704,856
Trainable params: 704,856
Non-trainable params: 0
_________________________________________________________________


In [50]:
print('Original English sentence: ')
print(english_sentences[:1][0])
print('\n')
print('Model translation: ')
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], fr_token))
print('\n')
print('Correct French translation: ')
print(french_sentences[:1][0])

Original English sentence: 
new jersey is sometimes quiet during autumn , and it is snowy in april .


Model translation: 
new jersey est parfois calme en mois et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Correct French translation: 
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [85]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """    
    # OPTIONAL: Implement
    # model architecture
    model = Sequential()
    # encoder
    model.add(LSTM(256, input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # compile model
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

In [86]:
# reshaping the input
tmp_x = pad(preproc_en_sentences, max_fr_seq_len)
tmp_x = tmp_x.reshape((-1, preproc_fr_sentences.shape[-2], 1))

In [87]:
# OPTIONAL: Train and Print prediction(s)
encdec_model = encdec_model(
    tmp_x.shape,
    max_fr_seq_len,
    en_vocab_size,
    fr_vocab_size
)

print(encdec_model.summary())

encdec_model.fit(
    tmp_x, 
    preproc_fr_sentences, 
    batch_size=1024, 
    epochs=10, 
    validation_split=0.2
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_49 (LSTM)               (None, 256)               264192    
_________________________________________________________________
repeat_vector_16 (RepeatVect (None, 21, 256)           0         
_________________________________________________________________
lstm_50 (LSTM)               (None, 21, 256)           525312    
_________________________________________________________________
dropout_19 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 21, 344)           88408     
Total params: 877,912
Trainable params: 877,912
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 26s 233us/step - los

In [88]:
print('Original English sentence: ')
print(english_sentences[:1][0])
print('\n')
print('Model translation: ')
print(logits_to_text(encdec_model.predict(tmp_x[:1])[0], fr_token))
print('\n')
print('Correct French translation: ')
print(french_sentences[:1][0])

Original English sentence: 
new jersey is sometimes quiet during autumn , and it is snowy in april .


Model translation: 
new jersey est jamais agréable en mois et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Correct French translation: 
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [34]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # model architecture        
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size, 
                        output_dim=english_vocab_size, 
                        input_length=input_shape[1], 
                        input_shape=input_shape[1:]))
    # encoder
    model.add(Bidirectional(GRU(output_sequence_length*10)))
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(Bidirectional(GRU(output_sequence_length*10, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size*10)))    
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # compile model
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

tests.test_model_final(model_final)
print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [35]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(
        x.shape,
        y.shape[1],
        len(x_tk.word_index),
        len(y_tk.word_index)
    )
    
    model.summary()
    
    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_en_sentences, preproc_fr_sentences, en_token, fr_token)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 15, 199)           39601     
_________________________________________________________________
bidirectional_39 (Bidirectio (None, 420)               516600    
_________________________________________________________________
repeat_vector_20 (RepeatVect (None, 21, 420)           0         
_________________________________________________________________
bidirectional_40 (Bidirectio (None, 21, 420)           795060    
_________________________________________________________________
dropout_20 (Dropout)         (None, 21, 420)           0         
_________________________________________________________________
time_distributed_39 (TimeDis (None, 21, 3440)          1448240   
_________________________________________________________________
time_distributed_40 (TimeDis (None, 21, 344)           1183704   
Total para

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [1]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 385196 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?